In [2]:
import os
import allel
import zarr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import sys
import dask
nnz = np.count_nonzero
import malariagen_data
ag3 = malariagen_data.Ag3("gs://vo_agam_release/")
import h5py
from allel import chunked

In [3]:
def log(*msg):
    print(' '.join(map(str, msg)), file=sys.stdout)
    sys.stdout.flush()

In [3]:
meta = pd.read_excel('../Data/Meta/meta_new_3_0.xlsx', sheet_name="meta3.0")
meta.head()

,sample_id,partner_sample_id,contributor,country,location,year,month,longitude,latitude,sex_call,...,pca_pops,pca_country,aim_species_fraction_colu_no2l,taxon,cohort_admin1_year,cohort_admin1_month,cohort_admin1_quarter,cohort_admin2_year,cohort_admin2_month,cohort_admin2_quarter
0,AB0085-Cx,BF2-4,Austin Burt,Burkina Faso,Pala,2012,7,-4.235,11.150,F,...,An.gam,BFgam,0.016488,gambiae,BF-09_gamb_2012,BF-09_gamb_2012_07,BF-09_gamb_2012_Q3,BF-09_Houet_gamb_2012,BF-09_Houet_gamb_2012_07,BF-09_Houet_gamb_2012_Q3
1,AB0086-Cx,BF2-6,Austin Burt,Burkina Faso,Pala,2012,7,-4.235,11.150,F,...,An.gam,BFgam,0.026316,gambiae,BF-09_gamb_2012,BF-09_gamb_2012_07,BF-09_gamb_2012_Q3,BF-09_Houet_gamb_2012,BF-09_Houet_gamb_2012_07,BF-09_Houet_gamb_2012_Q3
2,AB0087-C,BF3-3,Austin Burt,Burkina Faso,Bana,2012,7,-4.472,11.233,F,...,An.col,BFcol,0.981007,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_colu_2012_Q3,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07,BF-09_Houet_colu_2012_Q3
3,AB0088-C,BF3-5,Austin Burt,Burkina Faso,Bana,2012,7,-4.472,11.233,F,...,An.col,BFcol,0.987593,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_colu_2012_Q3,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07,BF-09_Houet_colu_2012_Q3
4,AB0089-Cx,BF3-8,Austin Burt,Burkina Faso,Bana,2012,7,-4.472,11.233,F,...,An.col,BFcol,0.973597,coluzzii,BF-09_colu_2012,BF-09_colu_2012_07,BF-09_colu_2012_Q3,BF-09_Houet_colu_2012,BF-09_Houet_colu_2012_07,BF-09_Houet_colu_2012_Q3


In [4]:
meta.pca_pops.unique()

array(['An.gam', 'An.col', 'gcx1-GM', 'gcx2', 'gcx1-GW'], dtype=object)

In [5]:
callset_fn = '../Data/Final/Data/chrom_3_west_final.h5'
callset = h5py.File(callset_fn, mode='r')

In [6]:
gt = allel.GenotypeDaskArray(callset['3']['calldata'])
gt

<GenotypeDaskArray shape=(160926, 1190, 2) dtype=int8>

In [9]:
callset_fn = '/media/randomx/Volume/Ag1000G/Phase3/Data/Extras/outgroup_allele_counts_phase3.h5'
callset_out = h5py.File(callset_fn, mode='r')

In [10]:
callset_out['3R'].keys()

<KeysViewHDF5 ['chri', 'epir']>

In [11]:
pos_3R = pd.read_csv('../Data/Final/Data/pos_3R_prun.csv', header=None).iloc[0].to_numpy() 
pos_3L = pd.read_csv('../Data/Final/Data/pos_3L_prun.csv', header=None).iloc[0].to_numpy() 

In [12]:
ac_chri_3R = callset_out['3R']['chri'][pos_3R]
ac_chri_3L = callset_out['3L']['chri'][pos_3L]
ac_chri = np.vstack((ac_chri_3R,ac_chri_3L))

In [13]:
subpops = {
    #'all': list(range(len(meta))),
    'An.gam': meta[meta.pca_pops == 'An.gam'].index.tolist(),
    'An.col': meta[meta.pca_pops == 'An.col'].index.tolist(),
    'gcx2': meta[meta.pca_pops == 'gcx2'].index.tolist(),
    'gcx1-GM': meta[meta.pca_pops == 'gcx1-GM'].index.tolist(),
    'gcx1-GW': meta[meta.pca_pops == 'gcx1-GW'].index.tolist(),
}

In [29]:
%%time
ac_subpops = gt.count_alleles_subpops(subpops, max_allele=1)

CPU times: user 21.2 ms, sys: 4.01 ms, total: 25.2 ms
Wall time: 24.9 ms


In [31]:
ac_subpops['chri'] = allel.AlleleCountsDaskArray(ac_chri[:,:2])
ac_subpops

{'An.gam': <AlleleCountsDaskArray shape=(160926, 2) dtype=int32>,
 'An.col': <AlleleCountsDaskArray shape=(160926, 2) dtype=int32>,
 'gcx2': <AlleleCountsDaskArray shape=(160926, 2) dtype=int32>,
 'gcx1-GM': <AlleleCountsDaskArray shape=(160926, 2) dtype=int32>,
 'gcx1-GW': <AlleleCountsDaskArray shape=(160926, 2) dtype=int32>,
 'chri': <AlleleCountsDaskArray shape=(160926, 2) dtype=int32>}

In [32]:
populations = ['chri','An.gam','An.col','gcx1-GM','gcx1-GW','gcx2']

In [33]:
acsu = dict()
for pop in populations:
    acsu[pop] = ac_subpops[pop][:, :2].compute()

In [34]:
def to_treemix(acs, fn):
    pops = sorted(acs.keys())
    n_variants = acs[pops[0]].shape[0]
    n_alleles = acs[pops[0]].shape[1]
    assert n_alleles == 2, 'only biallelic variants supported'
    for pop in pops[1:]:
        assert n_variants == acs[pop].shape[0], 'bad number of variants for pop %s' % pop
        assert n_alleles == acs[pop].shape[1], 'bad number of alleles for pop %s' % pop
        
    with open(fn, 'wt', encoding='ascii') as f:
        print(' '.join(pops), file=f)
        for i in range(n_variants):
            print(' '.join([','.join(map(str, acs[pop][i])) for pop in pops]), file=f)

In [35]:
outdir = 'Treemix/'
!mkdir -pv {outdir}
fn = os.path.join(outdir, 'treemix_fw_chri')
to_treemix(acsu, fn)
!gzip -fv {fn}

mkdir: directory '0531_new_treemix/' creata
0531_new_treemix/treemix_fw_chri:	 69.6% -- replaced with 0531_new_treemix/treemix_fw_chri.gz


In [4]:
df=pd.read_csv('Treemix/treemix_fw_chri.gz', sep=' ')
df

,An.col,An.gam,chri,gcx1-GM,gcx1-GW,gcx2
0,"748,0","856,42","0,0","154,0","186,0","394,0"
1,"361,387","234,664","0,0","66,88","78,108","192,202"
2,"748,0","890,6","0,0","150,4","169,17","391,1"
3,"666,80","894,1","0,0","154,0","184,2","370,22"
4,"730,16","792,104","0,0","154,0","185,1","391,1"
...,...,...,...,...,...,...
160921,"601,143","745,153","1,0","110,44","149,36","320,74"
160922,"675,68","778,114","1,0","145,8","166,18","362,29"
160923,"646,92","814,82","1,0","136,18","164,22","331,61"
160924,"734,14","893,3","0,0","147,7","186,0","393,1"


---------------------------------------

## Sandbox Treemix

In [12]:
callset_pass= ag3 = malariagen_data.Ag3("gs://vo_agam_release/")
allele_counts= zarr.open('../Data/Extras/fw_biallelic_count_country.zarr/')
outgroup_allele_counts= zarr.open('../Data/Extras/outgroup_alleles_phase3.zarr')

In [13]:
sel1 = np.array([True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True])
sel2 = np.array([True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,False,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,True,True,True,True,True,True,False,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,False,True,True,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True])

In [14]:
def outgroup_ascertainment(chrom, start, stop, outgroups):
    
    # locate region
    filter_pass = ag3.site_filters(chrom, mask="gamb_colu")
    f = filter_pass[:].compute()
    pos = ag3.snp_sites(chrom, field='POS')
    pos = allel.SortedIndex(pos[f].compute())
    locr = pos.locate_range(start, stop)
    
    # ascertain SNPs
    loca = np.zeros(pos.shape, dtype='b1')
    loca[locr] = True
    log('outgroup ascertainment, initial', nnz(loca))
    for s in outgroups:
        ac = allel.AlleleCountsArray(outgroup_allele_counts[chrom][s][:,:2])
        # non-missing
        locs = (ac.sum(axis=1) > 0)
        loca &= locs
        log(s, nnz(loca))
        
    return loca

In [15]:
def ingroup_ascertainment(chrom, start, stop, segpops):

    # locate region
    filter_pass = ag3.site_filters(chrom, mask="gamb_colu")
    f = filter_pass[:].compute()
    pos = ag3.snp_sites(chrom, field='POS')
    pos = allel.SortedIndex(pos[f].compute())
    locr = pos.locate_range(start, stop)

    # ascertain SNPs
    loca = np.zeros(pos.shape, dtype='b1')
    loca[locr] = True
    log('ingroup ascertainment, initial', nnz(loca))

    
    # require segregating
    for pop in segpops:
        ac = allel.AlleleCountsArray(allele_counts[chrom][pop][:])
        loc_seg = ac.min(axis=1) > 0
        loca &= loc_seg
        log('after require segregating in', pop, nnz(loca))
        
    return loca

In [16]:
def to_treemix(acs, fn):
    pops = sorted(acs.keys())
    n_variants = acs[pops[0]].shape[0]
    n_alleles = acs[pops[0]].shape[1]
    assert n_alleles == 2, 'only biallelic variants supported'
    for pop in pops[1:]:
        assert n_variants == acs[pop].shape[0], 'bad number of variants for pop %s' % pop
        assert n_alleles == acs[pop].shape[1], 'bad number of alleles for pop %s' % pop
        
    with open(fn, 'wt', encoding='ascii') as f:
        print(' '.join(pops), file=f)
        for i in range(n_variants):
            print(' '.join([','.join(map(str, acs[pop][i])) for pop in pops]), file=f)

In [25]:
def downsample_and_prune(chrom, start, stop, loc_asc,
                         n=100000, ldp_size=500, ldp_step=250, ldp_threshold=.1, ldp_n_iter=1):

    # all variant positions    
    filter_pass = ag3.site_filters(chrom, mask="gamb_colu")
    f = filter_pass[:].compute()
    pos = ag3.snp_sites(chrom, field='POS')
    pos = pos[f]
    pos = allel.SortedIndex(pos)
    posa = pos[loc_asc]

    # randomly downsample
    gt = ag3.snp_genotypes(chrom, sample_sets=["AG1000G-BF-A", "AG1000G-BF-B", "AG1000G-BF-C", "AG1000G-CI", "AG1000G-GH", "AG1000G-GM-A", "AG1000G-GM-B", 
                                       "AG1000G-GM-C", "AG1000G-GN-A", "AG1000G-GN-B", "AG1000G-ML-A", "AG1000G-ML-B", "AG1000G-GW"])
    genotype = allel.GenotypeDaskArray(gt).subset(sel0=f) 
    genotype = genotype.subset(sel1 = sel1)
    genotype = genotype.subset(sel1 = sel2)
    geno_subset = genotype.subset(sel0=loc_asc).compute()
    downsample_n = 100000
    
    if posa.size > downsample_n:
        print("Downsampling... to {0} from {1}".format(downsample_n, posa.size))
        idx = np.random.choice(np.arange(0, posa.size), downsample_n, False)
        idx.sort()
        pos = np.take(posa, idx)
        g = geno_subset.take(idx, axis=0)
    
    else:
        
        # skip downsampling
        g = geno_subset

 
    gn = g.to_n_alt()

    
    # prune    
    for i in range(ldp_n_iter):
        loc_unlinked = allel.locate_unlinked(gn, size=ldp_size, step=ldp_step, threshold=ldp_threshold)
        n = np.count_nonzero(loc_unlinked)
        n_remove = gn.shape[0] - n
        log('iteration', i+1, 'retaining', n, 'removing', n_remove, 'variants')
        gnu = gn.compress(loc_unlinked, axis=0)
        posu = pos.compress(loc_unlinked)
        locu = pos.locate_keys(posu)

    return locu

In [26]:
def run_analysis(rname, chrom, start, stop, outgroups, segpops,
                 n=100000, ldp_size=500, ldp_step=250, ldp_threshold=.1, ldp_n_iter=1):

    # initial ascertainment
    loc_og_asc = outgroup_ascertainment(chrom, start, stop, outgroups=outgroups)
    loc_ig_asc = ingroup_ascertainment(chrom, start, stop, segpops=segpops)
    loc_asc = loc_og_asc & loc_ig_asc
    log('initial ascertainment', nnz(loc_asc))
    
    # downsample and prune
    locu = downsample_and_prune(chrom, start, stop, loc_asc, 
                                n=n, ldp_size=ldp_size, ldp_step=ldp_step, 
                                ldp_threshold=ldp_threshold, ldp_n_iter=ldp_n_iter)
    
    # write allele counts
    acsu = dict()
    for pop in segpops:
        acsu[pop] = allele_counts[chrom][pop][:, :2][locu]
    for pop in outgroups:
        acsu[pop] = outgroup_allele_counts[chrom][pop][:, :2][locu]

    outdir = 'treemix/'
    !mkdir -pv {outdir}
    fn = os.path.join(outdir, '%s.new_treemix.txt' % rname)
    to_treemix(acsu, fn)
    !gzip -fv {fn}

In [27]:
outgroups = ['chri']
segpops = ['BFgam', 'BFcol', 'CIcol', 'GHcol', 'GHgam', 'GM-2', 'GM-1','GNgam', 'GNcol', 'MAgam', 'MAcol', 'GW', 'GWgam']
n = 100000
ldp_n_iter = 5
region_3R_24mbp = '3R-24Mbp', '3R', 1, 24_000_000
region_3L_free = '3L-free', '3L', 18_000_000, 41_000_000

In [28]:
rname, chrom, start, stop = region_3R_24mbp
log(rname, chrom, start, stop)
run_analysis(rname, chrom, start, stop, outgroups, segpops, n=n, ldp_n_iter=ldp_n_iter)

3R-24Mbp 3R 1 24000000
outgroup ascertainment, initial 18316117
chri 12088099
ingroup ascertainment, initial 18316117
after require segregating in BFgam 2506748
after require segregating in BFcol 1102387
after require segregating in CIcol 738367
after require segregating in GHcol 616605
after require segregating in GHgam 455698
after require segregating in GM-2 405818
after require segregating in GM-1 404126
after require segregating in GNgam 396964
after require segregating in GNcol 227928
after require segregating in MAgam 226308
after require segregating in MAcol 225389
after require segregating in GW 224217
after require segregating in GWgam 168342
initial ascertainment 93620
iteration 1 retaining 73990 removing 19630 variants
iteration 2 retaining 73990 removing 19630 variants
iteration 3 retaining 73990 removing 19630 variants
iteration 4 retaining 73990 removing 19630 variants
iteration 5 retaining 73990 removing 19630 variants
treemix/3R-24Mbp.new_treemix.txt:	 96.1% -- replace

In [29]:
rname, chrom, start, stop = region_3L_free
log(rname, chrom, start, stop)
run_analysis(rname, chrom, start, stop, outgroups, segpops, n=n, ldp_n_iter=ldp_n_iter)

3L-free 3L 18000000 41000000
outgroup ascertainment, initial 16952385
chri 10040109
ingroup ascertainment, initial 16952385
after require segregating in BFgam 2325501
after require segregating in BFcol 1002959
after require segregating in CIcol 667921
after require segregating in GHcol 559952
after require segregating in GHgam 410666
after require segregating in GM-2 366701
after require segregating in GM-1 365169
after require segregating in GNgam 358384
after require segregating in GNcol 208480
after require segregating in MAgam 206717
after require segregating in MAcol 205884
after require segregating in GW 204976
after require segregating in GWgam 156252
initial ascertainment 76858
iteration 1 retaining 60885 removing 15973 variants
iteration 2 retaining 60885 removing 15973 variants
iteration 3 retaining 60885 removing 15973 variants
iteration 4 retaining 60885 removing 15973 variants
iteration 5 retaining 60885 removing 15973 variants
treemix/3L-free.new_treemix.txt:	 97.0% -- re

--------------------------------

In [31]:
df_1=pd.read_csv('treemix/3R-24Mbp.new_treemix.txt.gz', sep=' ')
df_1

,BFcol,BFgam,CIcol,GHcol,GHgam,GM-1,GM-2,GNcol,GNgam,GW,GWgam,MAcol,MAgam,chri
0,"270,0","312,0","160,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
1,"270,0","314,0","160,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
2,"268,0","314,0","160,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
3,"268,0","314,0","160,0","126,0","72,0","393,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
4,"268,0","314,0","160,0","126,0","72,0","392,2","150,4","22,0","246,0","181,5","16,0","170,0","250,0","0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73985,"270,0","314,0","160,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
73986,"270,0","314,0","160,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
73987,"270,0","314,0","160,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
73988,"270,0","313,1","160,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"


In [32]:
df_2 = pd.read_csv('treemix/3L-free.new_treemix.txt.gz', sep=' ')
df_2

,BFcol,BFgam,CIcol,GHcol,GHgam,GM-1,GM-2,GNcol,GNgam,GW,GWgam,MAcol,MAgam,chri
0,"269,1","314,0","160,0","124,0","72,0","392,0","152,0","22,0","246,0","186,0","16,0","168,0","250,0","0,0"
1,"270,0","314,0","160,0","124,0","72,0","392,0","152,0","22,0","246,0","186,0","16,0","168,0","250,0","0,0"
2,"270,0","314,0","160,0","123,0","72,0","392,0","152,0","22,0","246,0","186,0","16,0","168,0","250,0","0,0"
3,"270,0","314,0","160,0","124,0","72,0","392,0","152,0","22,0","246,0","186,0","16,0","168,0","250,0","0,0"
4,"270,0","314,0","160,0","124,0","72,0","391,0","152,0","22,0","246,0","184,0","16,0","168,0","250,0","0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60880,"270,0","314,0","158,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
60881,"270,0","314,0","158,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
60882,"270,0","314,0","158,0","126,0","72,0","386,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
60883,"270,0","314,0","160,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"


In [33]:
frames = [df_1, df_2]
df = pd.concat(frames).reset_index().drop(columns=["index"])

In [34]:
df

,BFcol,BFgam,CIcol,GHcol,GHgam,GM-1,GM-2,GNcol,GNgam,GW,GWgam,MAcol,MAgam,chri
0,"270,0","312,0","160,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
1,"270,0","314,0","160,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
2,"268,0","314,0","160,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
3,"268,0","314,0","160,0","126,0","72,0","393,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
4,"268,0","314,0","160,0","126,0","72,0","392,2","150,4","22,0","246,0","181,5","16,0","170,0","250,0","0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134870,"270,0","314,0","158,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
134871,"270,0","314,0","158,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
134872,"270,0","314,0","158,0","126,0","72,0","386,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"
134873,"270,0","314,0","160,0","126,0","72,0","394,0","154,0","22,0","246,0","186,0","16,0","170,0","250,0","0,0"


In [35]:
df.to_csv('treemix/chrom_3_chri_pca_pops.txt.gz', sep = " ", index=False)